# Path planning

Duckietown exercises for Sept 30th, 2019


In this exercise, you will complete the implementation of a RRT planner allowing your Duckiebot to go from your current position to a goal position in a road with obstacles.

First, run this code. A window will pop-up with a top view of the environment your Duckiebot will go through. Apart from the loop, you should see some cones, a couple of lost Duckies and another Duckiebot on the road!

In [ ]:
%matplotlib 

import os, sys
import math
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.helpers import launch_env, wrap_env, view_results_ipython, change_exercise, force_done, topViewSimulator, load_env_obstacles
from simulation.gym_duckietown.wrappers import DiscreteWrapper, UndistortWrapper, SteeringToWheelVelWrapper
from simulation.gym_duckietown.simulator import Simulator
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint


### Setting up environment 
local_env = topViewSimulator()
local_env = launch_env(simclass = topViewSimulator, map_name ="loop_obstacles")
local_env = wrap_env(local_env)
local_env = SteeringToWheelVelWrapper(local_env)
local_env.reset()

### Bird view of environment
plt.figure(0)
obs, _, d, _ = local_env.step([0, 0])
plt.imshow(obs, cmap='gray')
plt.show()


Here, we will define the settings of the path planning problem: the list of obstacles and the goal position.

In this exercise, the obstacles are not moving. They are detected from the environment they are modelled as circular. Their info are given as such: \[x, y, diameter\]. Undrivable tiles are also represented by circular obstacles, so that the Duckiebot can only drive on the road.

In [ ]:
### Setting up the problem: list of obstacles and goal position
# Goal position
goal = [1,1]
# List of obstacles
list_obstacles = load_env_obstacles(local_env)
print('\n'.join('[{:f}, {:f}, {:f}]'.format(*obstacle) for obstacle in list_obstacles))

This function allows you to build a path from your current position to the goal position while avoiding obstacles. The path is a list of x-y positions going backwards from the end goal to the start position. The planner uses the RRT (Rapid Random Tree) algorithm that you have seen in class.

In this function, the `RRT_planner` class is called from your file `notebooks/code/exercise_03_path_planning/RRT.py`.

In [ ]:
from notebooks.code.exercise_03_path_planning.RRT import RRT_planner

def make_plan():
    ### Problem parameters
    # Start position
    start = local_env.cur_pos[0], local_env.cur_pos[2]
    # Area of exploration [x_min, x_max, y_min, y_max]
    rand_area = [0, 4.5, 0, 4.5]
    # No animation because we will call it many times in a row
    show_anim = False
    plt.figure(1)
    
    ### RRT Parameters
    # Distance of each step
    max_branch_length = 0.3
    # Resolution of the path (at which obstacles are checked)
    path_res = 0.1
    # Goal sampling rate : probability to artificially sample the goal
    goal_sample_rate=5
    # Max number of iterations
    max_iter = 10000
    
    ### Initializing and running the planner
    rrt_planner = RRT_planner(start, goal, list_obstacles, rand_area, max_branch_length, path_res, goal_sample_rate, max_iter)
    path = rrt_planner.plan(show_anim=show_anim)

    ### Finalization
    if path is None:
        print("Cannot find path")
    else:
        print("Found path!!")
        if True:
            rrt_planner.draw_graph(final_path = path)
    return path

This function will make the Duckiebot go to the next point of the path in two steps using proportional control: first, your robot turns towards the goal point, then, it moves straight towards it.

In [ ]:
def go_to_next_point(new_path):
    # Desired position is next point of the path
    next_pos = new_path[-2]
    # Current position and angle
    cur_pos_x = local_env.cur_pos[0]
    cur_pos_y = local_env.cur_pos[2]
    cur_angle = local_env.cur_angle

    ### Step 1: turn to reach the right angle
    # Angle that we need to reach
    x_diff = next_pos[0]-cur_pos_x
    y_diff = next_pos[1]-cur_pos_y
    angle_to_be = -1* np.angle(x_diff + y_diff * 1j) # Negative because of frame mismatch

    # Turning with P control
    while math.fabs(local_env.cur_angle%(2*np.pi) - angle_to_be%(2*np.pi)) > 0.05:
        v = 0.
        omega = (angle_to_be%(2*np.pi)-local_env.cur_angle%(2*np.pi) )
        obs, _, d, _ = local_env.step([v, omega])
        if d:
            print("Crash")
            break

    ### Step 2: move towards goal
    # Compute distance
    dist =  math.sqrt((local_env.cur_pos[0]-next_pos[0])**2 + (local_env.cur_pos[2]-next_pos[1])**2)
    last_dist = math.inf
    # Move forward until you reach the goal
    while dist > 0.02 and last_dist > dist:
        omega = 0
        v = dist
        obs, _, d, _ = local_env.step([v, omega])
        last_dist = dist
        dist = math.sqrt((local_env.cur_pos[0]-next_pos[0])**2 + (local_env.cur_pos[2]-next_pos[1])**2)
        if d:
            print("Crash")
            break

Finally, we're ready to go! Let's alternate both planning and driving until you reach the goal.

In [ ]:
%matplotlib 

dist_to_goal =  math.sqrt((local_env.cur_pos[0]-goal[0])**2 + (local_env.cur_pos[2]-goal[1])**2)

while dist_to_goal > 0.3:
    my_path = make_plan()
    if my_path is None:
        break
    go_to_next_point(my_path)
    dist_to_goal =  math.sqrt((local_env.cur_pos[0]-goal[0])**2 + (local_env.cur_pos[2]-goal[1])**2)



Ooops. Cannot find path? That's because the RRT planner is incomplete.

**Your task:** Finish implementing the `plan()` function of the `RRT_planner` class, which lives in `notebooks/code/exercise_03_path_planning/RRT.py` (l. 55 to 62)

Once you are done and the code above works (don't forget to run again the definition of `make_plan()` to enact your modifications), add the file **RRT.py** to your submission.

You can see what is happening here:

In [ ]:
local_env.close()
view_results_ipython(local_env)

**Question 1**

Is it needed to re-plan everytime the first path point is reached? In which situation would it be critical?

**Question 2**

Is this the most efficient method to reach the goal? What could you do to improve it?

**Question 3**

What would happen if the Duckiebot was placed in an environment with very small obstacles? Would RRT be able to avoid them? Which parameters would you need to adjust, and what would be the side effects?

Write your answers in a text file in your submission: **path-planning-answers.txt**